# Intuition behind TD vs. MC

From: https://towardsdatascience.com/temporal-difference-learning-47b4a7205ca8
<img src="13-intuition.png" style="width: 600px">

# Dynamic Programming, Monte Carlo (MC) and Temporal Difference (TD) Learning
Graphs from Katerina Fragkiadaki's notes on Temporal Difference Learning; you can find the full lecture notes on the google drive, called `12 - TD vs MC`.
<img src="13-DP.png" style="width: 500px">
<img src="13-MC.png" style="width: 500px">
<img src="13-TD.png" style="width: 500px">

# Introduction to SARSA

From https://www.youtube.com/watch?v=P9XezMuPfLE 
* Type of TD learning (so is Q-learning): you don't wait for the end of an episode to learn;
* Model free: does not need a complete model of the environment to get learning done;
* On policy method (_as opposed to Q-learning which is off-policy; we will not be going into details but watch the video to learn more!_);
* Bootstrapping: uses estimates to make more estimates, meaning that you don't need a lot of data to get moving.
* The algorithm
    - Initialize $\alpha$ (learning rate), $\gamma$ (discount value) and $\epsilon$ (how greedy is your algorithm?)
    - Initialize Q(s, a)
    - Initialize S
    - Choose A(S) using $\epsilon$ greedy from Q
    - Loop
        - Take action A, get reward, enter state S'
        - Choose A'(S') using $\epsilon$ greedy from A
        - Update Q: Q(s, a) $\rightarrow$ Q(s, a) + $\alpha$\[R + $\gamma$*Q(s',a')-Q(s,a)\]
        - S $\rightarrow$ S', A $\rightarrow$ A'

# The algorithm (from Reinforcement Learning by Sutton & Barto)

<img src="13-SARSA_alg.png">

# Some more insight from Jake Tuero (Edmonton TA)

* **State Values** and action-state values are always associated with a policy. The value of a state represents _"given that I have a policy and will continue to follow its recommendations, what is the value of being in a given state"_. The value comes from the expected reward that I will gain from starting in that state and following my policy.
* We generally like to work with **Action Values**, which is almost the same thing but with one additional condition: _"given that I am in a state and take action a, what is the expected reward I will gain by following my policy"_.
* Since we are generally concerned with what actions I should take, the **State-Action value** gives us this value directly, so we like to work with these. If I only had a value function, I would have to ask myself what is the state I would end up in after taking each respective action.
* The **State-Action value** is _q_ in the pseudocode above. This can be any function (neural network), but in our case it is linear. This linear function has a weight vector w, feature vector x, and computes the state-action value by np.dot(w,x).
* So given a _state s_ and _action a_, we use the code to get the feature vector, then dot it with the current weights to get the _state-action value_
* The **reward** is given by the environment step
* The reward also grounds our estimate values. An example would be _"what is the income I expect to make in my lifetime starting from today?"_. Lets say I predict its 1,000,000, so **q($s_{t}$, $a_{t}$)** = 1,000,000. I then take an action (one year of work) and get a reward (income) of 50,000. I then ask myself the same question _"what is the income I expect to make in my lifetime starting from today, which is actually 1 year in the future?"_. Since I have less years to work, my expected value should be less than 1,000,000. The difference between my previous estimate vs my current estimate + reward tells me how wrong I was about my predictions.
* The difference between **q($s_{t}$, $a_{t}$)** and **reward + q($s_{t+1}$, $a_{t+1}$)** is called the **TD error**. The larger the td-error is, the more we are wrong about our estimates, and so we update our weights in proportion to the magnitude of the td-error.
* The linear q allows for a simple update rule. Once we get into non-linear approximators like NNs, it gets a bit more complicated and the update rule will slightly change as we will let pytorch do the gradients for us and we have to rewrite the above into a loss function.


# Sarsa learning on Mountain car with linear function approximation

In this exercise you will be coding semi-gradient sarsa learning algorithm on the mountain car task. In the mountain car task, a car is stuck in the valley between two mountains and the goal is to steer it to the top of the right mountain. The car is underpowered to drive up the right mountain directly. Instead the car has to first move away from the goal and then, by applying full throttle it will build up enough inertia to reach the top of the right mountain. The agent's state space consists of position and velocity (continuous space). The agent can pick one of the three available actions (throttle forward, throttle backward, zero throttle). The reward for reaching the top of the right mountain is 0 and a reward of -1 is given to the agent everywhere else. You can find additional task description [here](https://gym.openai.com/envs/MountainCar-v0/). Complete the code to train RL agent solve this task.

## Tile coding

We use a tile coding to convert the continuous state space into a feature vector. For now, you can treat the below code as a black box and just think of it as a function that will provide you the features when you pass it a state and an action. In tile coding, the entire state space is divided into multiple grids. Given a state, the tile coder returns the indicies of the tiles corresponding to agent's location. We use multiple tiles with different offsets to obtain the agent's location. We get a binary feature vector corresponding to the agent's state.

# More on tile coding
From: https://www.youtube.com/watch?v=6DiCN_HmpxQ
<img src="13-tile_coding_1.png" style="width: 600px">
<img src="13-tile_coding_2.png" style="width: 600px">

Use the fact that the number of active tiles is always significantly less than the number total tiles to calculate the value function efficiently. All you need to do is sum weights of “active” features! In the example above, you get: $\hat{v}(s,w)$ = $w^{T}x(s)$ = 1.5 + 0.5 = 2 \
The principle remains the same when extrapolating this from a state-value function (v) to a state-action value function (q).\
In the above example, the **number of tiles is 8** and the **number of tiling is 2**.

In [1]:
"""
Tile Coding Software version 3.0beta
by Rich Sutton
based on a program created by Steph Schaeffer and others
External documentation and recommendations on the use of this code is available in the 
reinforcement learning textbook by Sutton and Barto, and on the web.
These need to be understood before this code is.

This software is for Python 3 or more.

This is an implementation of grid-style tile codings, based originally on
the UNH CMAC code (see http://www.ece.unh.edu/robots/cmac.htm), but by now highly changed. 
Here we provide a function, "tiles", that maps floating and integer
variables to a list of tiles, and a second function "tiles-wrap" that does the same while
wrapping some floats to provided widths (the lower wrap value is always 0).

The float variables will be gridded at unit intervals, so generalization
will be by approximately 1 in each direction, and any scaling will have 
to be done externally before calling tiles.

Num-tilings should be a power of 2, e.g., 16. To make the offsetting work properly, it should
also be greater than or equal to four times the number of floats.

The first argument is either an index hash table of a given size (created by (make-iht size)), 
an integer "size" (range of the indices from 0), or nil (for testing, indicating that the tile 
coordinates are to be returned without being converted to indices).
"""

basehash = hash

class IHT:
    "Structure to handle collisions"
    def __init__(self, sizeval):
        self.size = sizeval                        
        self.overfullCount = 0
        self.dictionary = {}

    def __str__(self):
        "Prepares a string for printing whenever this object is printed"
        return "Collision table:" + \
               " size:" + str(self.size) + \
               " overfullCount:" + str(self.overfullCount) + \
               " dictionary:" + str(len(self.dictionary)) + " items"

    def count (self):
        return len(self.dictionary)
    
    def fullp (self):
        return len(self.dictionary) >= self.size
    
    def getindex (self, obj, readonly=False):
        d = self.dictionary
        if obj in d: return d[obj]
        elif readonly: return None
        size = self.size
        count = self.count()
        if count >= size:
            if self.overfullCount==0: print('IHT full, starting to allow collisions')
            self.overfullCount += 1
            return basehash(obj) % self.size
        else:
            d[obj] = count
            return count

def hashcoords(coordinates, m, readonly=False):
    if type(m)==IHT: return m.getindex(tuple(coordinates), readonly)
    if type(m)==int: return basehash(tuple(coordinates)) % m
    if m==None: return coordinates

from math import floor, log
from itertools import zip_longest

def tiles (ihtORsize, numtilings, floats, ints=[], readonly=False):
    """returns num-tilings tile indices corresponding to the floats and ints"""
    qfloats = [floor(f*numtilings) for f in floats]
    Tiles = []
    for tiling in range(numtilings):
        tilingX2 = tiling*2
        coords = [tiling]
        b = tiling
        for q in qfloats:
            coords.append( (q + b) // numtilings )
            b += tilingX2
        coords.extend(ints)
        Tiles.append(hashcoords(coords, ihtORsize, readonly))
    return Tiles

def tileswrap (ihtORsize, numtilings, floats, wrapwidths, ints=[], readonly=False):
    """returns num-tilings tile indices corresponding to the floats and ints, wrapping some floats"""
    qfloats = [floor(f*numtilings) for f in floats]
    Tiles = []
    for tiling in range(numtilings):
        tilingX2 = tiling*2
        coords = [tiling]
        b = tiling
        for q, width in zip_longest(qfloats, wrapwidths):
            c = (q + b%numtilings) // numtilings
            coords.append(c%width if width else c)
            b += tilingX2
        coords.extend(ints)
        Tiles.append(hashcoords(coords, ihtORsize, readonly))
    return Tiles

num_tiles = 2048 # number of tiles
num_tilings = 8

iht = IHT(num_tiles)

def get_features(state, action):
  """
  Returns the features 
  """
  tile_id = tiles(iht, num_tilings, [num_tilings*state[0]/(0.6+1.2), num_tilings*state[1]/(0.07+0.07)], [action]) # return the indices of the tiles corresponding to agent's location
  features = np.zeros(feature_length)
  features[tile_id] = 1 # set features corresponding to the above indicies to 1
  return features

In [2]:
import gym
import numpy as np

OpenAI gym is a toolkit that has many commonly used reinforcement learning environments. [check](https://gym.openai.com/docs/) this out for a detailed instructions on how to set it up and use it.

We will make use of the following commands:


```
env = gym.make(env_name) # sets up the environment
c_s = env.reset() # resets the environment and returns the starting state for the agent
n_s, rew, done, _ = env.step(action) # applies action in the current state and returns the next state, reward. done is true if the episode has ended.
```



In [3]:
Tenv = gym.make('MountainCar-v0') # make gym environment

We will now initialize our FA weights and set the values of hyperparameters.

In [4]:
'''
Fill up the hyperparameters here
'''
feature_length=num_tiles
weights = np.zeros(feature_length) # this is our FA weights, initialized to 0
epsilon =0.1
# amount of exploration in your policy
num_episodes = 100 # total number of episodes to train the agent
    # note: episode != steps!
lr_rate = epsilon/num_tilings # learning rate to update the weights (divided by num_tilings to account for tilings)
discount = 0.99 # discount factor (gamma)

In [5]:
'''
This is our policy. An epsilon greedy policy that return a random action during exploration and samples an action corresponding to max action value during exploitation.
'''
def get_action(state):
    if np.random.binomial(1, epsilon) == 1:
        # return random action - exploration
        return np.random.choice([0,1,2]) # [0,1,2] are the available actions to the agent
    values = []
    for action in [0,1,2]: #loop for each step of episode
        values.append(weights.T.dot(get_features(state, action)))
    # return action corresponding to the max action value - exploitation
    return np.random.choice([action_ for action_, value_ in enumerate(values) if value_ == np.max(values)])

Let's go through 1 single step first (recall that we already defined Tenv):

In [6]:
s_0=Tenv.reset()
print(f"Starting state: {s_0}")
a_1=get_action(s_0)
print(f"First action: {a_1}")
next_s, reward, done, _ = Tenv.step(a_1)
print(f" next state:{next_s} \n reward: {reward} \n Is this episode done? {done}")

Starting state: [-0.56860934  0.        ]
First action: 1
 next state:[-5.68272785e-01  3.36554283e-04] 
 reward: -1.0 
 Is this episode done? False


Refer to Section 10.1 from [Sutton and Barto book](http://www.incompleteideas.net/book/RLbook2020.pdf) to fill up the following code.

## Answer the following questions:


1.   Plot the training curves i.e., plot the returns obtained per episode in the y-axis and the episode number in the x-axis.
2.   What happens if you set epsilon to 0 while training? Also experiment with different values of epsilon (0.1, 0.5). What do you observe?
3.   What happens if you change the learning rate?


In [ ]:
'''
Fill up the following code.
'''
for epi in range(num_episodes):
    env = gym.make('MountainCar-v0') # initialize env
    cur_state=env.reset()
    cur_action=get_action(s_0)
    done = False
    while not done:
        next_state, reward, done, _ = env.step(cur_action) # get next_state, reward, done, _ from env
        next_action=get_action(next_state) # get next_action
    # get features corresponding to the current state and the next state
    # compute action values of the current and next states
    # compute td error
    # update weights
    # update current state and current action

In [ ]:
'''
Write the plot code here
'''